In [ ]:
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

model = SamModel.from_pretrained("facebook/sam-vit-base")
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
model.to("cuda")

raw_image = Image.open("../1_media/input_images/cat.jpg")
input_points = [[[450, 600]]] 


In [ ]:
inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to("cuda")
outputs = model(**inputs)
masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())
scores = outputs.iou_scores

In [ ]:
mask_image = masks[0].squeeze(0)
mask_image = (mask_image * 255).cpu().numpy()
mask_image = mask_image.astype("uint8").transpose(1, 2, 0)
mask_image = Image.fromarray(mask_image)
mask_image.save("cat_mask.jpg")

In [ ]:
from transformers import pipeline

In [ ]:
generator =  pipeline("mask-generation", device = 2, points_per_batch = 256)
image_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
outputs = generator(image_url, points_per_batch = 256)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    

plt.imshow(np.array(raw_image))
ax = plt.gca()
for mask in outputs["masks"]:
    show_mask(mask, ax=ax, random_color=True)
plt.axis("off")
plt.show()


In [ ]:
from transformers import pipeline

generator = pipeline("mask-generation", model="Zigeng/SlimSAM-uniform-50", points_per_batch=64, device="cuda")
image_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
outputs = generator(image_url)


In [ ]:
len(outputs["masks"])

In [ ]:
outputs["masks"][46].shape

In [ ]:
import torch
from PIL import Image

In [ ]:
masks = outputs["masks"]
mask_image = masks[0]
mask_image = (mask_image * 255)
mask_image = mask_image.astype("uint8")
mask_image = Image.fromarray(mask_image)
mask_image.save("cat_mask.jpg")